<a href="https://colab.research.google.com/github/financieras/math_for_ai/blob/main/228_ejemplo_del_taxista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# El misterio del "1" en Machine Learning

Una de las cosas que más confunde a quienes aprenden Machine Learning es **¿de dónde sale ese "1" que multiplicamos por el bias?**

Vamos a resolverlo con un ejemplo muy intuitivo: calculando el precio de la carrera de un taxi.

---

## El problema: Tarifas de taxi

Un taxi cobra según esta fórmula:

$$\text{Precio} = 2.5 + 1.3 \times \text{km}$$

Es una línea recta:
$$y = b + m x$$
O bien, en notación de Machine Learning, la recta es:

$$y = w_0 + w_1 x$$

Donde:
- **2.5€** es la **bajada de bandera** (coste fijo, independiente de los km). En la recta es el término independiente, intercepto o bias.
- **1.3€/km** es el precio por kilómetro recorrido. En la recta es la pendiente o coeficiente de la variable independiente $x$.

### La pregunta clave

¿Cómo expresamos esto en forma matricial para calcular **múltiples carreras a la vez**?

El problema es que la bajada de bandera (2.5€) **no depende de ninguna variable**. Es un término constante. Pero en una multiplicación matricial, necesitamos multiplicar cada peso por algo.

**Solución:** Multiplicamos el término constante por 1.

$$\text{Precio} = 2.5 \times \boxed{1} + 1.3 \times \text{km}$$

Este "1" es lo que nos permite incluir el término independiente en la notación matricial.

---

## Caso 1: Una sola carrera

Empecemos calculando el precio de una carrera de 10 km usando notación matricial.

In [4]:
import numpy as np

# Una carrera: [1, km]
# El 1 es para multiplicar por la bajada de bandera
x = np.array([1, 10])  # [bias, kilómetros]

# Pesos: [bajada_bandera, precio_por_km]
w = np.array([2.5, 1.3])

# Calculamos el precio
precio = x @ w  # Equivalente a: x.dot(w)

print(f"Carrera de {x[1]} km")
print(f"Precio: {precio}€")
print(f"\nDesglose:")
print(f"  Bajada de bandera: {w[0]} × {x[0]}  =  {w[0] * x[0]}€")
print(f"  Kilómetros:        {w[1]} × {x[1]} = {w[1] * x[1]}€")
print(f"  Total:                        {precio}€")

Carrera de 10 km
Precio: 15.5€

Desglose:
  Bajada de bandera: 2.5 × 1  =  2.5€
  Kilómetros:        1.3 × 10 = 13.0€
  Total:                        15.5€


### Notación matricial

Para una sola observación:

$$\hat{y} = \mathbf{x}^T \mathbf{w} = \begin{bmatrix} 1 & 10 \end{bmatrix} \begin{bmatrix} 2.5 \\ 1.3 \end{bmatrix} = 15.5$$

**Dimensiones:** $\mathbf{x}^T_{1 \times 2} \cdot \mathbf{w}_{2 \times 1} = \hat{y}$ (escalar)

---

## Caso 2: Múltiples carreras (Batch Processing)

En Machine Learning, **nunca procesamos datos de uno en uno**. Siempre trabajamos con **batches** (lotes) de datos.

Supongamos que tenemos 4 carreras diferentes:

| Carrera | Kilómetros |
|---------|------------|
| 1       | 10         |
| 2       | 4          |
| 3       | 25         |
| 4       | 20         |

Queremos calcular los 4 precios **simultáneamente**.

In [6]:
# Kilómetros de 4 carreras
kilometros = np.array([10, 4, 25, 20])

# Construimos la matriz X: cada FILA es una carrera
# Añadimos columna de unos para la bajada de bandera
X = np.column_stack([np.ones(4), kilometros])

# Pesos
w = np.array([2.5, 1.3])

# Calculamos los 4 precios a la vez
precios = X @ w

print("Matriz X (cada fila es una carrera):")
print(X)
print("\nVector de pesos w:")
print(w)
print("\nPrecios de cada carrera:")
for i, precio in enumerate(precios, 1):
    print(f"  Carrera {i} ({X[i-1, 1]:.0f} km): {precio}€")
print(f"\nTotal facturado:     {np.sum(precios)}€")

Matriz X (cada fila es una carrera):
[[ 1. 10.]
 [ 1.  4.]
 [ 1. 25.]
 [ 1. 20.]]

Vector de pesos w:
[2.5 1.3]

Precios de cada carrera:
  Carrera 1 (10 km): 15.5€
  Carrera 2 (4 km): 7.7€
  Carrera 3 (25 km): 35.0€
  Carrera 4 (20 km): 28.5€

Total facturado:     86.7€


### Notación matricial para batch

$$\hat{\mathbf{y}} = \mathbf{X} \mathbf{w} = \begin{bmatrix}
1 & 10 \\
1 & 4 \\
1 & 25 \\
1 & 20
\end{bmatrix}
\begin{bmatrix}
2.5 \\
1.3
\end{bmatrix}
=
\begin{bmatrix}
15.5 \\
7.7 \\
35.0 \\
28.5
\end{bmatrix}$$

**Dimensiones:** $\mathbf{X}_{4 \times 2} \cdot \mathbf{w}_{2 \times 1} = \hat{\mathbf{y}}_{4 \times 1}$

**Observa:**
- Cada **fila** de $\mathbf{X}$ es una observación (una carrera)
- La **primera columna** contiene solo unos (para el bias)
- Obtenemos **4 predicciones** con una sola operación matricial

---

## Caso 3: Múltiples características

Ahora añadimos una segunda variable: **minutos de espera** (coste: 0.5€/min)

$$\text{Precio} = 2.5 + 1.3 \times \text{km} + 0.5 \times \text{min}$$

Es la ecuación de un plano:
$$y = w_0 + w_1 x_1 + w_2 x_2$$

### Una carrera con dos características

In [12]:
# Una carrera: 10 km y 15 minutos de espera
x = np.array([1, 10, 15])  # [bias, km, minutos]

# Pesos: [bajada_bandera, precio_km, precio_minuto]
w = np.array([2.5, 1.3, 0.5])

# Calculamos el precio
precio = x @ w

print(f"Carrera: {x[1]} km, {x[2]} min")
print(f"Precio: {precio}€")
print(f"\nDesglose:")
print(f"  Bajada de bandera: {w[0]} × {x[0]}  =  {w[0] * x[0]}€")
print(f"  Kilómetros:        {w[1]} × {x[1]} = {w[1] * x[1]}€")
print(f"  Minutos:           {w[2]} × {x[2]} =  {w[2] * x[2]}€")
print(f"  Total:                        {precio}€")

Carrera: 10 km, 15 min
Precio: 23.0€

Desglose:
  Bajada de bandera: 2.5 × 1  =  2.5€
  Kilómetros:        1.3 × 10 = 13.0€
  Minutos:           0.5 × 15 =  7.5€
  Total:                        23.0€


### Batch de 4 carreras con dos características

Ahora tenemos:

| Carrera | Kilómetros | Minutos |
|---------|------------|----------|
| 1       | 10         | 15       |
| 2       | 4          | 0        |
| 3       | 25         | 5        |
| 4       | 20         | 10       |

In [13]:
# Datos de las 4 carreras
kilometros = np.array([10, 4, 25, 20])
minutos = np.array([15, 0, 5, 10])

# Construimos X: cada fila es una carrera [1, km, min]
X = np.column_stack([np.ones(4), kilometros, minutos])

# Pesos: [bajada_bandera, precio_km, precio_minuto]
w = np.array([2.5, 1.3, 0.5])

# Calculamos los 4 precios
precios = X @ w

print("Matriz X (cada fila es una carrera):")
print("  [bias, km, min]")
print(X)
print("\nVector de pesos w:")
print("  [bajada, €/km, €/min]")
print(w)
print("\nPrecios calculados:")
for i in range(len(precios)):
    print(f"  Carrera {i+1} ({X[i,1]:.0f} km, {X[i,2]:.0f} min): {precios[i]}€")
print(f"\nTotal facturado: {np.sum(precios)}€")

Matriz X (cada fila es una carrera):
  [bias, km, min]
[[ 1. 10. 15.]
 [ 1.  4.  0.]
 [ 1. 25.  5.]
 [ 1. 20. 10.]]

Vector de pesos w:
  [bajada, €/km, €/min]
[2.5 1.3 0.5]

Precios calculados:
  Carrera 1 (10 km, 15 min): 23.0€
  Carrera 2 (4 km, 0 min): 7.7€
  Carrera 3 (25 km, 5 min): 37.5€
  Carrera 4 (20 km, 10 min): 33.5€

Total facturado: 101.7€


### Notación matricial

$$\hat{\mathbf{y}} = \mathbf{X} \mathbf{w} = \begin{bmatrix}
1 & 10 & 15 \\
1 & 4 & 0 \\
1 & 25 & 5 \\
1 & 20 & 10
\end{bmatrix}
\begin{bmatrix}
2.5 \\
1.3 \\
0.5
\end{bmatrix}
=
\begin{bmatrix}
23.0 \\
7.7 \\
37.5 \\
33.5
\end{bmatrix}$$

**Dimensiones:** $\mathbf{X}_{4 \times 3} \cdot \mathbf{w}_{3 \times 1} = \hat{\mathbf{y}}_{4 \times 1}$

---

## 🎯 Conexión con Machine Learning

### Regresión Lineal

Podemos ver el paralelismo con la regresión lineal, que para $n$ características se expresa como un hiperplano:

$$\hat{y} = w_0 + w_1 x_1 + w_2 x_2 + \cdots + w_n x_n$$

En forma matricial:

$$\hat{\mathbf{y}} = \mathbf{X} \mathbf{w}$$

Donde:
- $\mathbf{X}$: matriz de características $(m \times (n+1))$
  - $m$ = número de observaciones (muestras)
  - $n$ = número de features (características)
  - La **primera columna** son unos (para el bias $w_0$)
- $\mathbf{w}$: vector de pesos $((n+1) \times 1)$
- $\hat{\mathbf{y}}$: vector de predicciones $(m \times 1)$

### Ejemplo con scikit-learn

In [17]:
from sklearn.linear_model import LinearRegression

# Datos (sin incluir columna de unos, sklearn lo hace automáticamente)
X_train = np.array([[10, 15],
                    [4, 0],
                    [25, 5],
                    [20, 10]])
y_train = np.array([23.0, 7.7, 37.5, 33.5])

# Entrenar modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

print("Parámetros aprendidos por sklearn:")
print(f"  Bias (w₀): {modelo.intercept_:.12f}")
print(f"  Coeficientes (w₁, w₂): {modelo.coef_}")

print("\nComparando con nuestros pesos originales:")
print(f"  w₀ (bajada): {w[0]}")
print(f"  w₁ (€/km):   {w[1]}")
print(f"  w₂ (€/min):  {w[2]}")

# Predecir nueva carrera: 30 km, 8 minutos
nueva_carrera = np.array([[30, 8]])
precio_predicho = modelo.predict(nueva_carrera)[0]
precio_manual = 2.5 + 1.3*30 + 0.5*8

print(f"\nNueva carrera (30 km, 8 min):")
print(f"  Predicción sklearn: {precio_predicho:.12f}€")
print(f"  Cálculo manual:     {precio_manual}€")

Parámetros aprendidos por sklearn:
  Bias (w₀): 2.500000000000
  Coeficientes (w₁, w₂): [1.3 0.5]

Comparando con nuestros pesos originales:
  w₀ (bajada): 2.5
  w₁ (€/km):   1.3
  w₂ (€/min):  0.5

Nueva carrera (30 km, 8 min):
  Predicción sklearn: 45.500000000000€
  Cálculo manual:     45.5€


---

## 🧠 Ventajas del procesamiento en batch

El procesamiento en batch (múltiples observaciones a la vez) es **fundamental** en Machine Learning:

### 1. **Eficiencia computacional**
Procesar 100 muestras simultáneamente es **muchísimo más rápido** que procesarlas una por una.

In [24]:
import time

# Generamos 1 millón de carreras aleatorias
n_carreras = 1_000_000
X_grande = np.column_stack([
    np.ones(n_carreras),
    np.random.uniform(1, 50, n_carreras),  # km
    np.random.uniform(0, 30, n_carreras)   # minutos
])

# Método 1: Una por una (loop)
start = time.time()
precios_loop = []
for i in range(n_carreras):
    precio = X_grande[i] @ w
    precios_loop.append(precio)
tiempo_loop = time.time() - start

# Método 2: Todas a la vez (vectorizado)
start = time.time()
precios_vectorizado = X_grande @ w
tiempo_vectorizado = time.time() - start

print(f"Procesando {n_carreras:,} carreras:")
print(f"  Método loop (una por una): {tiempo_loop*1000:.2f} ms")
print(f"  Método vectorizado (batch): {tiempo_vectorizado*1000:.2f} ms")
print(f"  Speedup: {tiempo_loop/tiempo_vectorizado:.1f}x más rápido")

Procesando 1,000,000 carreras:
  Método loop (una por una): 3063.47 ms
  Método vectorizado (batch): 4.01 ms
  Speedup: 764.4x más rápido


### 2. **Aprovechamiento del hardware**

Las GPUs están optimizadas para operaciones matriciales grandes. Por eso los frameworks de Deep Learning (PyTorch, TensorFlow) trabajan siempre con batches.

### 3. **Estabilidad en entrenamiento**

En redes neuronales, calcular gradientes sobre un batch es más estable que sobre una sola muestra.

Los tamaños de batch típicos para entrenamiento son: 32, 64, 128, 256, ..., para optimizar los productos matriciales en las tarjetas gráficas.

---

## 📐 Fórmula general

Para $n$ características (features) y $m$ observaciones (muestras):

### Una observación:

$$\hat{y} = \mathbf{x}^T \mathbf{w} = \begin{bmatrix} 1 & x_1 & x_2 & \cdots & x_n \end{bmatrix} \begin{bmatrix} w_0 \\ w_1 \\ w_2 \\ \vdots \\ w_n \end{bmatrix}$$

**Dimensiones:** $\mathbf{x}^T_{1 \times (n+1)} \cdot \mathbf{w}_{(n+1) \times 1} = \hat{y}$ (escalar)

### Batch de $m$ observaciones:

$$\hat{\mathbf{y}} = \mathbf{X} \mathbf{w} = \begin{bmatrix}
1 & x_{11} & x_{12} & \cdots & x_{1n} \\
1 & x_{21} & x_{22} & \cdots & x_{2n} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & x_{m1} & x_{m2} & \cdots & x_{mn}
\end{bmatrix}
\begin{bmatrix}
w_0 \\ w_1 \\ w_2 \\ \vdots \\ w_n
\end{bmatrix}$$

**Dimensiones:** $\mathbf{X}_{m \times (n+1)} \cdot \mathbf{w}_{(n+1) \times 1} = \hat{\mathbf{y}}_{m \times 1}$

---

En este artículo hemos realizado un recorrido por los motivos que hacen interesante trabajar de forma matricial con lotes de datos. Esto muestra el origen del misterioso **1** que aparece en las fórmulas matriciales.